install packagees

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


prepare/process data
 - normalize numerical col
 - create one hot encoding
 - create feature matric

In [2]:
data = pd.read_csv("cleaned_data.csv")
user_data = pd.read_csv("user_data.csv")
#print(data.head())

# Convert 'startYear' to numeric if it's not already
data['startYear'] = pd.to_numeric(data['startYear'], errors='coerce')

# Handle missing values in numerical columns by filling with the mean or median
numerical_cols = ['startYear', 'averageRating', 'numVotes']
for col in numerical_cols:
    if data[col].isnull().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)
        #print(f"Filled missing values in {col} with median.")

# For categorical columns, fill missing values with a placeholder or mode
categorical_cols = ['titleType', 'genres', 'directorNames', 'writerNames', 'isAdult']
for col in categorical_cols:
    data[col].fillna('Unknown', inplace=True)
    #print(f"Filled missing values in {col} with 'Unknown'.")

data = data[data['startYear'] > 1990]
#print(data.head())
#print(user_data.head())

/var/folders/1y/4sm5vl2d7_xdd0jwk0tgnhn00000gn/T/ipykernel_1790/57310068.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna('Unknown', inplace=True)


In [3]:
# movie data

# Initialize the scaler
scaler = MinMaxScaler()

# Select numerical columns
numerical_cols = ['startYear', 'averageRating', 'numVotes']

# Fit and transform the data
data = data.copy()
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

print("Numerical columns normalized successfully!")
data[numerical_cols].head()

# One-Hot Encode 'titleType' and 'isAdult'
data = pd.get_dummies(data, columns=['titleType', 'isAdult'], prefix=['titleType', 'isAdult'])

# 0 and 1 assignment for titleType_movie, isAdult_0, isAdult_1
data[["titleType_movie", "isAdult_0", "isAdult_1"]] = data[["titleType_movie", "isAdult_0", "isAdult_1"]].astype(int)

# genre multi-encoding
data['genre_list'] = data['genres'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# mlb encoder
multi_label_encoder = MultiLabelBinarizer()
genres_encoded = multi_label_encoder.fit_transform(data['genre_list'])
genres_encoded_df = pd.DataFrame(genres_encoded, columns=multi_label_encoder.classes_)
data = pd.concat([data, genres_encoded_df], axis=1)


#print(user_data.head())

# director label encoding
label_encoder = LabelEncoder()
data['directorNames'] = label_encoder.fit_transform(data['directorNames'])

# writer label encoding
data['writerNames'] = label_encoder.fit_transform(data['writerNames'])

# movie_id using label encoding
data["primaryTitle"] = label_encoder.fit_transform(data["primaryTitle"])


data.head()
#print(data.columns)
#user_data.head()





Numerical columns normalized successfully!


,tconst,primaryTitle,startYear,genres,directorNames,writerNames,averageRating,numVotes,titleType_movie,isAdult_0,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
2595,tt0015414,86996,0.272727,[],74346,141434,0.488889,0.000004,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2685,tt0015724,38456,0.060606,"['Drama', 'Mystery', 'Romance']",31896,40258,0.588889,0.000009,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
14035,tt0035423,81539,0.303030,"['Comedy', 'Fantasy', 'Romance']",44598,125169,0.600000,0.030575,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15926,tt0038086,137074,0.060606,['Thriller'],39191,49947,0.666667,0.000007,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33106,tt0062336,167731,0.878788,['Drama'],86031,108933,0.611111,0.000064,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
#user_data

user_numerical_cols = ['UserRating']

user_data[user_numerical_cols] = scaler.fit_transform(user_data[user_numerical_cols])

user_data['user_genre_list'] = user_data['FavoriteGenres'].apply(lambda x: eval(x) if isinstance(x, str) else x)
user_genres_encoded = multi_label_encoder.fit_transform(user_data['user_genre_list'])
user_genres_encoded_df = pd.DataFrame(user_genres_encoded, columns=multi_label_encoder.classes_)
user_data = pd.concat([user_data, user_genres_encoded_df], axis=1)


# director label encoding
label_encoder = LabelEncoder()
user_data['FavoriteDirectors'] = label_encoder.fit_transform(user_data['FavoriteDirectors'])

# writer label encoding
user_data['FavoriteActors'] = label_encoder.fit_transform(user_data['FavoriteActors'])

# movie_id label encoding
user_data["primaryTitle"] = label_encoder.fit_transform(user_data["primaryTitle"])

# user_id label ecoding
user_data["UserID"] = label_encoder.fit_transform(user_data["UserID"])


user_data.head()
#print(user_data.columns)

,UserID,tconst,primaryTitle,UserRating,FavoriteGenres,FavoriteActors,FavoriteDirectors,user_genre_list,Action,Adult,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
0,0,tt0000009,149138,0.000000,['Romance'],149,19868,[Romance],0,0,...,0,0,0,1,0,0,0,0,0,0
1,111111,tt0000147,224595,0.444444,"['News', 'Documentary', 'Sport']",49,46569,"[News, Documentary, Sport]",0,0,...,0,1,0,0,0,1,0,0,0,0
2,222222,tt0000502,35350,0.888889,[],299,105924,[],0,0,...,0,0,0,0,0,0,0,0,0,0
3,252417,tt0000574,247271,0.777778,['Action'],249,33864,[Action],1,0,...,0,0,0,0,0,0,0,0,0,0
4,263528,tt0000591,242575,0.888889,['Drama'],144,89998,[Drama],0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
genre_list = [col for col in data.columns if col not in ['tconst', 'primaryTitle', 'startYear', 'genres', 'directorNames',
       'writerNames', 'averageRating', 'numVotes', 'titleType_movie',
       'isAdult_0', 'isAdult_1', 'genre_list']]

movie_features = pd.concat([data[['tconst', 'averageRating', 'numVotes']]],axis=1)

movie_features['genres_list'] = data[genre_list].apply(
    lambda genres: [1 if genres[genre] == 1 else 0 for genre in genre_list], axis=1  # Access genre columns by name
)

# formats user features in an array
user_features = pd.concat([user_data[['tconst', 'UserID', 'UserRating']]],axis=1)

# merges user and movie data into an array
merged_data = pd.merge(user_features, movie_features, on='tconst', how='inner')

merged_data.head()


,tconst,UserID,UserRating,averageRating,numVotes,genres_list
0,tt0015414,177334,1.000000,0.488889,0.000004,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,tt0015724,187334,0.222222,0.588889,0.000009,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,tt0035423,44847,0.444444,0.600000,0.030575,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,tt0038086,65857,0.111111,0.666667,0.000007,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,tt0062336,244761,0.777778,0.611111,0.000064,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [6]:
x_input = []

for index, row in merged_data.iterrows():
    x_input.append(list(row['genres_list']) + [row['averageRating'], row['numVotes'], row['UserRating']])

x_input[0]


[0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0.4888888888888889,
 4.057523510812793e-06,
 1.0]

create vae model design
 - encoder
 - decoder
 - define loss function

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions

class Encoder(nn.Module):

    def __init__( self, input_dim, hidden_dim1, hidden_dim2, latent_dim ):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc_mu = nn.Linear(hidden_dim2, latent_dim)      # Mean of latent distribution
        self.fc_log_var = nn.Linear(hidden_dim2, latent_dim) # Log variance of latent distribution

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return mu, log_var  # Return mean and log variance

class Decoder(nn.Module):

    def __init__( self, input_dim, hidden_dim1, hidden_dim2, latent_dim ):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim2)
        self.fc2 = nn.Linear(hidden_dim2, hidden_dim1)
        self.fc_out = nn.Linear(hidden_dim1, input_dim)

    def forward(self, z):
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        # might want to change .softmax to .sigmoid
        return F.sigmoid(self.fc_out(z))  # Return class probabilities

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim1, hidden_dim2, latent_dim)
        self.decoder = Decoder(input_dim, hidden_dim1, hidden_dim2, latent_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)  # Standard deviation
        epsilon = torch.randn_like(std)  # Random noise
        return mu + epsilon * std  # Reparameterization trick

    def forward(self, x):
        mu, log_var = self.encoder(x)  # Encode input to latent distribution
        z = self.reparameterize(mu, log_var)  # Sample latent variable
        x_reconstructed = self.decoder(z)  # Decode back to input space
        return x_reconstructed, mu, log_var  # Return reconstructed data, mean, and log variance

def loss_function(recon_x, x, mu, logvar):
    # Reconstruction Loss (Binary Cross Entropy)
    recon_loss = F.binary_cross_entropy(recon_x.view(-1), x.view(-1), reduction='sum')

    # KL Divergence Loss
    # KL = -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Combined Loss
    loss = recon_loss + kl_loss
    return loss


Training VAE
 - train model
  - forward pass
  - compute loss
  - backpropogation
  - run for # of epochs

In [8]:
# Hyperparameters
HIDDEN_DIM1 = 128
HIDDEN_DIM2 = 64
LATENT_DIM = 32

# Prepare data for training
merged_data_tensor = torch.tensor(x_input, dtype=torch.float32)
data_loader = DataLoader(TensorDataset(merged_data_tensor), batch_size=64, shuffle=True)

# Model setup
input_dim = merged_data_tensor.shape[1]
vae = VAE(input_dim, HIDDEN_DIM1, HIDDEN_DIM2, LATENT_DIM)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
EPOCHS = 20
vae.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in data_loader:
        batch = batch[0]  # Unpack the TensorDataset
        optimizer.zero_grad()

        # Forward pass
        recon_batch, mu, logvar = vae(batch)
        loss = loss_function(recon_batch, batch, mu, logvar)

        # Backward pass
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(data_loader.dataset):.4f}")

print("VAE training complete!")

Epoch 1, Loss: 4.6012
Epoch 2, Loss: 4.4025
Epoch 3, Loss: 4.3571
Epoch 4, Loss: 4.3346
Epoch 5, Loss: 4.3255
Epoch 6, Loss: 4.3239
Epoch 7, Loss: 4.3194
Epoch 8, Loss: 4.3190
Epoch 9, Loss: 4.3165
Epoch 10, Loss: 4.3109
Epoch 11, Loss: 4.3134
Epoch 12, Loss: 4.3117
Epoch 13, Loss: 4.3133
Epoch 14, Loss: 4.3090
Epoch 15, Loss: 4.3059
Epoch 16, Loss: 4.3081
Epoch 17, Loss: 4.3042
Epoch 18, Loss: 4.3043
Epoch 19, Loss: 4.3038
Epoch 20, Loss: 4.3043
VAE training complete!


Generate Latent Representations
- evaluate user preferences
- compare with other movies

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def get_user_embeddings(user_rating, movie_features, vae_model):
    vae_model.eval()
    with torch.no_grad():
        input_vector = torch.tensor(list(movie_features) + [user_rating], dtype=torch.float32)
        _, mu, _ = vae_model(input_vector)
    return mu.numpy()

def reccomend_movies(user_rating, movie_features, movie_embeddings, data, top_n=5):
    user_embedding = get_user_embeddings(user_rating, list(movie_features), vae)

    similarity_scores = cosine_similarity(user_embedding, movie_embeddings)

    top_indices = np.argsort(similarity_scores.flatten())[::-1][:top_n]

    recommended_movies = data.iloc[top_indices]

    recommended_movies['similarity_score'] = similarity_scores.flatten()[top_indices]

    return recommended_movies[['tconst', 'primaryTitle', 'averageRating', 'similarity_score']]

Content-Based Recomendations
 - Compute Similarities
 - generate reccomendations


In [10]:
movie_emeddings = []

vae.eval()
with torch.no_grad():
    for index, row in movie_features.iterrows():
        movie_input = torch.tensor(row['genres_list'] + [row['averageRating'], row['numVotes'], 0.5], dtype=torch.float32)
        mu, _ = vae.encoder(movie_input)
        movie_emeddings.append(mu.numpy())

user_rating = 4.5
movie_example_features = [0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0.5,
 6.057523510812793e-06,
 1.0]
movie_emeddings = np.vstack(movie_emeddings)
top_movies = reccomend_movies(user_rating, movie_example_features, movie_emeddings, data, top_n=5)
print(top_movies)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 29x128)

Testing and Validations
 - Evaluate Reccomendations
 - Fine Tune Model